In [58]:
import cv2
import numpy as np

In [59]:
def people_counter(gray):
    global i, centroid
    counts = []
    prev_centroids = []
    prev_directions = []

    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    rects, _ = hog.detectMultiScale(gray, winStride=(4, 4), padding=(8, 8), scale=1.05)

    for i, (x, y, w, h) in enumerate(rects):
        centroid = (int(x + w/2), int(y + h/2))
        
        # Draw contour and centroid on frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, centroid, 5, (0, 0, 255), -1)    

        # Check if previous centroid exists for this contour
        if len(prev_centroids) > i and prev_centroids[i] is not None:
            # Check if current centroid has crossed line 1
            if (prev_centroids[i][0] <= line_pos < centroid[0]) or (
                    prev_centroids[i][0] >= line_pos > centroid[0]):
                # Check if current centroid has crossed line 2
                if (prev_centroids[i][0] <= line_pos2 < centroid[0]) or (
                        prev_centroids[i][0] >= line_pos2 > centroid[0]):
                    # Check if centroid crossed from left to right
                    if prev_centroids[i][0] < line_pos and centroid[0] > line_pos2:
                        counts[i] += 1
                        break
                    # Check if centroid crossed from right to left
                    elif prev_centroids[i][0] > line_pos2 and centroid[0] < line_pos:
                        counts[i] -= 1
                        break
                    
                    
            

        # Add new contour's centroid and direction to the lists
        else:
            prev_directions.append(None)
            prev_centroids.append(centroid)
            counts.append(0)



In [60]:
def frame_processing():
    global contours
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray

In [61]:
def display_frame():
    # Draw lines on frame
    cv2.line(frame, (line_pos, 0), (line_pos, frame.shape[0]), (0, 0, 255), 2)
    cv2.line(frame, (line_pos2, 0), (line_pos2, frame.shape[0]), (255, 0, 0), 2)
    # Draw count for each contour on frame
    cv2.putText(frame, "Count: {}".format(sum(counts)), (10, 30 + 30 * i), font, font_scale, font_color, line_type)
    # Display frame
    cv2.imshow("Train Counter", frame)

In [62]:
def background_subtraction(frame):
    fgmask = fgbg.apply(frame)
    return cv2.bitwise_and(frame, frame, mask=fgmask)

In [63]:
def __main__():
    global frame
    while True:
        # Read frame from video capture
        ret, frame = cap.read()
        # Check if frame was successfully read
        if not ret:
            print("Error reading frame from video stream")
            break
        # Apply background subtraction
        # frame = background_subtraction(frame)
        # fgbg = cv2.createBackgroundSubtractorMOG2()
        
        # Convert frame to grayscale and apply Gaussian blur
        # gray = frame_processing()
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        
        # Process each contour
        people_counter(gray)
        # Display frames
        display_frame()
        # Wait for key press and limit frame rate to 15 fps
        if cv2.waitKey(33) & 0xFF == ord('q'):
            break
        # Update previous centroid
        prev_centroid = centroid
    # Release video capture and destroy all windows
    cap.release()
    cv2.destroyAllWindows()

In [64]:
cv2.startWindowThread()

# Create video capture object
cap = cv2.VideoCapture("../test_footage.mp4")

# Define font settings for text overlay
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)
line_type = 2

# Initialize variables
prev_directions = []
prev_centroids = []
counts = []
line_pos = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2)
line_pos2 = int((cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2)+75)
__main__()